# Семинар 3 - низкоуровневый ввод-вывод и файлы

Сегодня в программе:
1. Сисколы
2. Низкоуровневый ввод/вывод
3. Файлы и их друзья
4. Жесткие и символические ссылки
5. Директории
6. Fuse

Как программа может взаимодействовать с внешним миром? Одни из простейших примеров: ввод-вывод через терминал. На самом деле любое(*) взаимодействие программы с внешним миром - это сискол. 

Сискол - это взаимодействие программы с ядром операционной системы. Операционная система предоставляет программам ограниченное число сисколов, которые они могут использовать.

Примеры:
|**Действие**|**Сискол**|
|:----------:|:-------------:|
|ввод|read|
|вывод|write|
|динамическое выделение памяти|brk, mmap|
|создание нового процесса|fork|
|открытие сетевого соединения|connect|

Чтобы посмотреть, какие сисколы делает программа, есть утилита `strace`. Вернемся к примеру с Hello, world с первого семинара:

In [3]:
!cat helloworld.c
!gcc helloworld.c -o helloworld.out

#include <stdio.h>

int main() {
    printf("Hello, world!");
}


In [5]:
!strace ./helloworld.out

zsh:1: command not found: strace


```
execve("./a.out", ["./a.out"], 0xffffe11b47c0 /* 24 vars */) = 0
brk(NULL)                               = 0xbcf1f8633000
mmap(NULL, 8192, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0xeb50b6b61000
faccessat(AT_FDCWD, "/etc/ld.so.preload", R_OK) = -1 ENOENT (No such file or directory)
openat(AT_FDCWD, "/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
fstat(3, {st_mode=S_IFREG|0644, st_size=22297, ...}) = 0
mmap(NULL, 22297, PROT_READ, MAP_PRIVATE, 3, 0) = 0xeb50b6b5b000
close(3)                                = 0
openat(AT_FDCWD, "/lib/aarch64-linux-gnu/libc.so.6", O_RDONLY|O_CLOEXEC) = 3
read(3, "\177ELF\2\1\1\3\0\0\0\0\0\0\0\0\3\0\267\0\1\0\0\0(%\2\0\0\0\0\0"..., 832) = 832
fstat(3, {st_mode=S_IFREG|0755, st_size=1788672, ...}) = 0
mmap(NULL, 1957936, PROT_NONE, MAP_PRIVATE|MAP_ANONYMOUS|MAP_DENYWRITE, -1, 0) = 0xeb50b694a000
mmap(0xeb50b6950000, 1892400, PROT_READ|PROT_EXEC, MAP_PRIVATE|MAP_FIXED|MAP_DENYWRITE, 3, 0) = 0xeb50b6950000
munmap(0xeb50b694a000, 24576)           = 0
munmap(0xeb50b6b1f000, 36912)           = 0
mprotect(0xeb50b6afa000, 77824, PROT_NONE) = 0
mmap(0xeb50b6b0d000, 20480, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_FIXED|MAP_DENYWRITE, 3, 0x1ad000) = 0xeb50b6b0d000
mmap(0xeb50b6b12000, 49200, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_FIXED|MAP_ANONYMOUS, -1, 0) = 0xeb50b6b12000
close(3)                                = 0
set_tid_address(0xeb50b6b61fd0)         = 1961
set_robust_list(0xeb50b6b61fe0, 24)     = 0
rseq(0xeb50b6b626c0, 0x20, 0, 0xd428bc00) = 0
mprotect(0xeb50b6b0d000, 12288, PROT_READ) = 0
mprotect(0xbcf1ed22f000, 4096, PROT_READ) = 0
mprotect(0xeb50b6b67000, 8192, PROT_READ) = 0
prlimit64(0, RLIMIT_STACK, NULL, {rlim_cur=8192*1024, rlim_max=RLIM64_INFINITY}) = 0
munmap(0xeb50b6b5b000, 22297)           = 0
fstat(1, {st_mode=S_IFCHR|0620, st_rdev=makedev(0x88, 0), ...}) = 0
getrandom("\x7c\x8a\x7d\x05\xf9\xae\xdc\x44", 8, GRND_NONBLOCK) = 8
brk(NULL)                               = 0xbcf1f8633000
brk(0xbcf1f8654000)                     = 0xbcf1f8654000
write(1, "Hello World\n", 12Hello World
)           = 12
exit_group(0)                           = ?
+++ exited with 0 +++
```

Нас интересует предпоследний сискол - `write`. Изучим его подробнее.

Для этого воспользуемся утилитой `man` (manual). Рекомендуется освоится с ней, потому что она будет полезна всю оставшуюся часть курса. Синтаксис: `man <section> <search>`

```
MANUAL SECTIONS
    The standard sections of the manual include:

    1      User Commands
    2      System Calls
    3      C Library Functions
    4      Devices and Special Files
    5      File Formats and Conventions
    6      Games et. al.
    7      Miscellanea
    8      System Administration tools and Daemons

    Distributions customize the manual section to their specifics,
    which often include additional sections.
```

In [6]:
!man 2 write

WRITE(2)                      System Calls Manual                     WRITE(2)

NAME
     pwrite, write, pwritev, writev – write output

LIBRARY
     Standard C Library (libc, -lc)

SYNOPSIS
     #include <unistd.h>

     ssize_t
     pwrite(int fildes, const void *buf, size_t nbyte, off_t offset);

     ssize_t
     write(int fildes, const void *buf, size_t nbyte);

     #include <sys/uio.h>

     ssize_t
     writev(int fildes, const struct iovec *iov, int iovcnt);

     ssize_t
     pwritev(int fildes, const struct iovec *iov, int iovcnt, off_t offset);

DESCRIPTION
     write() attempts to write nbyte of data to the object referenced by the
     descriptor fildes from the buffer pointed to by buf.  writev() performs
     the same action, but gathers the output data from the iovcnt buffers
     specified by the members of the iov array: iov[0], iov[1], ...,
     iov[iovcnt-1].  pwrite() and pwritev() perform the same functions, but
     write to the specified position in the file wi

Видим определение (из linux): `ssize_t write(int fd, const void buf[count], size_t count);`

`buf` - указатель, из которого копировать данные. `count` - количество байт для записи. `fd` - файловый дескриптор.

Файловый дескриптор - это числовой идентификатор открытого файла в рамках одного процесса. Примеры стандартных файловых дескрипторов: `0 stdin, 1 stdout, 2 stderr`.

<img src="fd_table.gif" alt="file descriptors" width="800"/>

Как с этим работать?

Для открытия файла есть сискол `open`:
```c
int open(const char *path, int flags, ... /* mode_t mode */);
// flags O_RDONLY, O_WRONLY, ..., O_CREAT
```

Основные параметры открытия файлов:

* `O_RDONLY` - только для чтения;
* `O_WRONLY` - только на запись;
* `O_RDWR` - чтение и запись;
* `O_APPEND` - запись в конец файла;
* `O_TRUNC` - обнуление файла при открытии;
* `O_CREAT` - создание файла, если не существует;
* `O_EXCL` - создание файла только если он не существует.

В случае успеха возвращается неотрицательное число - дескриптор, в случае ошибки - значение `-1`.

Если выставлен флаг `O_CREAT`, обязательным параметром является набор POSIX-аттрибутов доступа к файлу. Как правило, они кодируются в восьмеричной системе исчисления в виде `0ugo`, где `u` - права доступа для владельца файла, `g` - права доступа для всех пользователей группы файла, `o` - для остальных.

В восьмеричной записи значения от 0 до 7 соответствуют комбинации трёх бит, где `r` это чтение, `w` запись, а `x` исполнение:
```c
00: ---
01: --x
02: -w-
03: -wx
04: r--
05: r-x
06: rw-
07: rwx
```

У `open` есть парный сискол: `close`. 

```c
int close(int fd);
```

Он возвращает `0` в случае успеха и `-1` при ошибке. Для всех открытых файловых дескрипторов должен быть вызван соответствующий `close`.

Для чтения и записи есть сисколы `read` и ранее упомянутый `write`:
```c
ssize_t read(int fd, void *buf, size_t count);
ssize_t write(int fd, const void *buf, size_t count);
// count максимальне количество байт, должно быть меньше размера буффера.
```

Возвращают они количество прочитанных/записанных байт. Гарантируется, что в случае успеха они обработают ненулевое число байт, но необязательно все доступные данные. Поэтому читать/писать нужно в цикле.

Начнем рассматривать примеры:

In [221]:
!cat basic_io.c
!gcc basic_io.c -o basic_io.out

#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>

int main() {
    int input_fd  = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/input.txt",  O_RDONLY);
    int output_fd = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/output.txt", O_WRONLY | O_TRUNC);

    int buffer_size = 16;
    char* buffer = (char*)malloc(buffer_size);

    int read_bytes = read(input_fd, buffer, buffer_size);
    int written = write(output_fd, buffer, read_bytes);

    close(input_fd);
    close(output_fd);

    return 0;
}

In [222]:
!./basic_io.out

Пример работает. Посмотрим на вывод `strace` для него:

In [15]:
!strace ./basic_io.out

zsh:1: command not found: strace


```
...
openat(AT_FDCWD, "/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/input.txt", O_RDONLY) = 3
openat(AT_FDCWD, "/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/output.txt", O_WRONLY|O_TRUNC) = 4
getrandom("\x87\xe6\xdb\xee\xd3\x40\x4d\xae", 8, GRND_NONBLOCK) = 8
brk(NULL)                               = 0xc599de5fc000
brk(0xc599de61d000)                     = 0xc599de61d000
read(3, "Hello, world!", 16)            = 13
write(4, "Hello, world!", 13)           = 13
close(3)                                = 0
close(4)                                = 0
exit_group(0)                           = ?
+++ exited with 0 +++
```

Все упомянутые в коде сисколы были исполнены, это наглядно видно.

Как можно улучшить текущий код? Вспомним, что `read` и `write` возвращают количество обработанных байт, которое может быть меньше, чем количество запрошенных байт. Учтем это:

In [19]:
!cat sized_io.c
!gcc sized_io.c -o sized_io.out

#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>

int main() {
    int input_fd  = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/input.txt",  O_RDONLY);
    int output_fd = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/output.txt", O_WRONLY | O_TRUNC);

    int buffer_size = 16;
    char* buffer = (char*)malloc(buffer_size);
    
    ssize_t read_during_iteration;
    size_t read_bytes = 0;
    do {
        read_during_iteration = read(input_fd, buffer + read_bytes, buffer_size - read_bytes);
        if (read_during_iteration > 0) {
            read_bytes += read_during_iteration;
        }
    } while (read_during_iteration != 0);

    ssize_t written_during_iteration;
    size_t written_bytes = 0;
    do {
        written_during_iteration = write(output_fd, buffer + written_bytes, read_bytes - written_bytes);
        if (written_during_iteration > 0) {
            written_bytes += written_during_iteration;
        }
    } while 

In [223]:
!./sized_io.out

Ничего не сломалось, это хорошо.

Можно ли улучшить программу еще? Да! Добавим обработку ошибок.

О том, какие ошибки могут возвращать `read` и `write`, можно почитать в man-е. Нас интересует тот факт, что некоторые из этих ошибок не являются фатальными и можно попробовать повторить неудавшееся действие. А именно, нас интересуют ошибки `EINTR` и `EAGAIN`. При возникновении ошибок сисколы возвращают `-1` и проставляют информацию об ошибке в специальную переменную `errno`.

In [21]:
!cat error_handling_io.c 
!gcc error_handling_io.c -o error_handling_io.out 

#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>
#include <errno.h>

int main() {
    int input_fd  = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/input.txt",  O_RDONLY);
    int output_fd = open("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/output.txt", O_WRONLY | O_TRUNC);

    int buffer_size = 16;
    char* buffer = (char*)malloc(buffer_size);
    
    ssize_t read_during_iteration;
    size_t read_bytes = 0;
    do {
        read_during_iteration = read(input_fd, buffer + read_bytes, buffer_size - read_bytes);
        if ((read_during_iteration < 0) && (errno != EINTR) && (errno != EAGAIN)) {
            return -1;
        }
        read_bytes += read_during_iteration;
    } while (read_during_iteration != 0);

    ssize_t written_during_iteration;
    size_t written_bytes = 0;
    do {
        written_during_iteration = write(output_fd, buffer + written_bytes, read_bytes - written_bytes);
        if ((written_during_iteration < 

In [224]:
!./error_handling_io.out

### Пример, демонстрирующий, что обработка таких ошибок играет значение:

In [225]:
!cat simple_reprinter.c 
!gcc simple_reprinter.c -o simple_reprinter.out 

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[]) {
    char buffer[4096];
    int bytes_read = read(STDIN_FILENO, buffer, sizeof(buffer));
    printf("From stdin: '%.*s'\n", bytes_read, buffer); // buffer not zero-terminated string after `read`!
    
    return 0;
}

In [226]:
!(echo -n "Hello, " ; sleep 1 ; echo -n "world!" 2>/dev/null) | ./simple_reprinter.out 

From stdin: 'Hello, '


Вывели не весь текст, потому что на момент вызова `read` он был не полностью доступен. Теперь с ретраями:

In [28]:
!cat smart_reprinter.c 

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <errno.h>

int read_retry(int fd, char* data, int size) {
    char* cdata = data;
    while (1) {
        int read_bytes = read(fd, cdata, size);
        if (read_bytes == 0) { // если read вернул 0, значит файловый дескриптор закрыт с другого конца 
                               // или конец файла
            return cdata - data;
        }
        if (read_bytes < 0) { // если возвращено значение < 0, то это ошибка
            if (errno == EAGAIN || errno == EINTR) { // она может быть retryable
                continue;
            } else { // а может быть критичной, и нет смысла пытаться повторить попытку чтения
                return -1;
            }
        }
        // если возвращенное значение > 0, значит успешно прочитано столько байт
        cdata += read_bytes;
        size -= read_bytes;
        if (size == 0) {
            return cdata - data;
        }
    }
}


Еще раз проверим, что просто `read` не дает нужного результата:

In [227]:
!gcc smart_reprinter.c -D_USE_READ -o smart_reprinter.out
!(echo -n "Hello," ; sleep 1 ; echo -n "world!" 2>/dev/null) | ./smart_reprinter.out 

Read 'Hello,'

С ретраями:

In [228]:
!gcc smart_reprinter.c -D_USE_READ_RETRY -o smart_reprinter.out
!(echo -n "Hello," ; sleep 1 ; echo -n "world!" 2>/dev/null) | ./smart_reprinter.out 

Read 'Hello,world!'

Как ведет себя `scanf`?🤔

In [229]:
!gcc smart_reprinter.c -D_USE_SCANF -o smart_reprinter.out
!(echo -n "Hello," ; sleep 1 ; echo -n "world!" 2>/dev/null) | ./smart_reprinter.out 

Read 'Hello,world!'

Видим, что `scanf` молодец. Не забываем ретраить такие ошибки.

Какие есть еще полезности:
1. `pread/pwrite` - чтение и запись по смещению
2. `readv/writev` - работа с несколькими буферами одновременно 
3. `cmd1 9>&1 cmd2` - "связали" файловый дескриптор 1 (stdout) cmd2 и файловый дескриптор 9 cmd1. Можно использовать с любыми fd

### О производительности

Возьмем файл, в котором 1m строк и программу, которая будет читать их и отправлять в stdout.

In [230]:
import random
import string

chars_per_line = 5

with open("huge_file.txt", 'w') as output:
    lines = (''.join(random.sample(string.ascii_lowercase, chars_per_line)) for _ in range(1_000_000))
    print("\n".join(lines), file=output)

In [231]:
!head -n 3 huge_file.txt

nohva
fdoxe
dtqfp


In [97]:
!cat perf.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>

void printf_impl(FILE* source) {
    char* line = NULL;
    size_t len = 0;
    ssize_t read;
    while ((read = getline(&line, &len, source)) != -1) {
        printf("%s", line);
    }

    if (line) {
        free(line);
    }
}

void write_impl(FILE* source) {
    char* line = NULL;
    size_t len = 0;
    ssize_t read;
    while ((read = getline(&line, &len, source)) != -1) {
        write(STDOUT_FILENO, line, len);
    }

    if (line) {
        free(line);
    }
}

int main() {
    FILE* file = fopen("/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/huge_file.txt", "r");

#if defined(_USE_PRINTF)
    printf_impl(file);
#endif
#if defined(_USE_WRITE)
    write_impl(file);
#endif

    fclose(file);
    
    return 0;
}


In [232]:
!gcc perf.c -D_USE_PRINTF -o perf.out 
!time ./perf.out > /dev/null

./perf.out > /dev/null  0.07s user 0.01s system 20% cpu 0.390 total


In [233]:
!gcc perf.c -D_USE_WRITE -o perf.out 
!time ./perf.out > /dev/null

./perf.out > /dev/null  0.15s user 2.43s system 93% cpu 2.765 total


> Наша реализация получилась медленнее в 15 раз. Почему?

**Ответ**: сисколы в целом - дорогая операция. Для того, чтобы она совершилась, процесс ставится на паузу и исполнение передается ядру. Это все занимает много времени. Поэтому глобальная идея: сисколы довольно непроизводительны, и если важен performance, можно рассмотреть возможность уменьшения их количества. Стандартные функции ввода/вывода из этих соображений буферизуют данные.


Количество сисколов `write` в программе с `printf`:
```c
(strace -e write ./perf.out > /dev/null) 2>&1 >/dev/null | wc -l
1465
```

И в программе с ручными вызовами `write`:
```c
(strace -e write ./perf.out > /dev/null) 2>&1 >/dev/null | wc -l
1000000
```

Напишем свой буфферизованный вывод

In [219]:
!cat buffered_perf.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>
#include <errno.h>

int write_retry(int fd, char* data, int size) {
    char* cdata = data;
    while (1) {
        int written_bytes = write(fd, cdata, size);
        if (written_bytes == 0) { // если write вернул 0, значит файловый дескриптор закрыт с другого конца 
                                  // или конец файла
            return cdata - data;
        }
        if (written_bytes < 0) { // если возвращено значение < 0, то это ошибка
            if (errno == EAGAIN || errno == EINTR) { // она может быть retryable
                continue;
            } else { // а может быть критичной, и нет смысла пытаться повторить попытку чтения
                return -1;
            }
        }
        // если возвращенное значение > 0, значит успешно прочитано столько байт
        cdata += written_bytes;
        size -= written_bytes;
        if (size == 0) {
            return cdata - data;
  

In [220]:
!gcc buffered_perf.c -o buffered_perf.out
!time ./buffered_perf.out > /dev/null

./buffered_perf.out > /dev/null  0.03s user 0.00s system 18% cpu 0.177 total


```c
(strace -e write ./perf.out > /dev/null) 2>&1 >/dev/null | wc -l
6
```

### Курсор

Когда вы читаете/пишите в файлы через файловые дескрипторы, операционная система поддерживает текущую позицию "курсора" - того места, с которым вы работаете. Чтобы читать/писать без оглядки на курсор, можно использовать `pread/pwrite`. Для работы непосредственно с курсором есть `lseek`.

In [205]:
!cat lseek_demo.c
!gcc lseek_demo.c -o lseek_demo.out

#include <assert.h>
#include <errno.h>
#include <fcntl.h>
#include <stdio.h>
#include <unistd.h>

int main() {
    int fd = open("input.txt", O_RDONLY);
    
    lseek(fd, 7, SEEK_SET);
    
    char buff[1];
    read(fd, &buff, sizeof(buff));

    write(STDOUT_FILENO, buff, sizeof(buff));
    close(fd);

    return 0;
}



In [206]:
!./lseek_demo.out

w

## Подробнее о файлах

Что такое файл?

.

.

.

🌈Everything is a file🌈

<img src="everything_is_a_file.webp" alt="everything_is_a_file" width="500"/>

Идея в том, что в Linux применен архитектурный подход, согласно которому очень многие вещи представлены в виде сущностей, схожих с файлами. Например, директории, процессы, внешние устройства, сокеты, ...

* Everything is a file
* Everything is a file descriptor
* Everything is a stream of bytes
* Everything has a file
* ...

Если всё - это файл, то что такое файл? Остановимся на том, что файл - это то, у чего есть `inode`. 

inode - сущность в ядре, в которой хранится вся (мета)информация о файле (название и само содержимое файла там отсутствуют).

<img src="inode_ext4.jpg" alt="inode" width="600"/>

Часть информации о ней можно извлечь в виде структуры `stat` через одноименный системный вызов:

```c
struct stat {
   dev_t     st_dev;         /* ID of device containing file */
   ino_t     st_ino;         /* Inode number */
   mode_t    st_mode;        /* File type and mode */
   nlink_t   st_nlink;       /* Number of hard links */
   uid_t     st_uid;         /* User ID of owner */
   gid_t     st_gid;         /* Group ID of owner */
   dev_t     st_rdev;        /* Device ID (if special file) */
   off_t     st_size;        /* Total size, in bytes */
   blksize_t st_blksize;     /* Block size for filesystem I/O */
   blkcnt_t  st_blocks;      /* Number of 512B blocks allocated */

   struct timespec st_atim;  /* Time of last access */
   struct timespec st_mtim;  /* Time of last modification */
   struct timespec st_ctim;  /* Time of last status change */

   /* Backward compatibility */
   #define st_atime st_atim.tv_sec      
   #define st_mtime st_mtim.tv_sec
   #define st_ctime st_ctim.tv_sec
};
```


```c
int (l)stat(const char *file_name, struct stat *stat_buffer)
```

`inode` - абстракция уровня файловой системы. Внутри одной файловой системы номера `inode` (`st_ino`) уникальны.

Посмотрим на устройство этой структуры на примере поля `.st_mode`: там лежит число, в бинарном представлении которого зашита информация о типе файлах и правах доступа. 

Биты соответствующие маскам:

`0170000` (`0b1111000000000`) - тип файла.

Эти биты стоит рассматривать как одно число, по значению которого можно определить тип файла, сравнивая это число с:

```c
S_IFSOCK 0140000 socket
S_IFLNK  0120000 symbolic link
S_IFREG  0100000 regular file
S_IFBLK  0060000 block device
S_IFDIR  0040000 directory
S_IFCHR  0020000 character device
S_IFIFO  0010000 FIFO
```

`0777` (`0b111111111`) - права на файл.

> Эти биты можно рассматривать как независимые биты, каджый из которых отвечает за право (пользователя, его группы, всех остальных) (читать/писать/выполнять) файл.

Попробуем извлечь какую-нибудь информация о файле через `stat`:

In [168]:
!cat basic_stat.c 
!gcc basic_stat.c -o basic_stat.out

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[]) {
    assert(argc > 1);

    struct stat s;
    int result = stat(argv[1], &s);
    if (result != 0) {
        printf("stat failed\n");
        return 1;
    }
    printf("is regular: %s    ", ((s.st_mode & S_IFMT) == S_IFREG) ? "yes" : "no "); // can use predefined mask
    printf("is directory: %s    ", S_ISDIR(s.st_mode) ? "yes" : "no "); // or predefined macro
    printf("is symbolic link: %s\n", S_ISLNK(s.st_mode) ? "yes" : "no "); 
    return 0;
}

In [110]:
!./basic_stat.out /Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/huge_file.txt

is regular: yes    is directory: no     is symbolic link: no 


In [111]:
!./basic_stat.out /Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs

is regular: no     is directory: yes    is symbolic link: no 


Что такое ссылка и символическая ссылка? Как следует из названия, это "ссылки" на другой файл

(Жесткая) ссылка на файл - это файл, имеющий под собой ту же самую `indode`, что и оригинальный файл. Таким образом, при изменении названия/содержимого файлов, эти изменения будут отражены и в оригинальном файле, и в (жесткой) ссылке на него. При удалении файла содержимое удаляется, если количество жестких ссылок стало равно 0.

Символическая ссылка на файл - это файл, имеющий свою собственную `inode`, внутри которого указан *путь* к оригинальному файлу. При работе с символической ссылкой происходит её резолвинг с последующим доступом к оригинальному файлу. Если оригинальный файл будет удален/перемещен, символическая ссылка станет "провисшей". 

<img src="links.jpg" alt="hard & symbolic" width="600"/>

Поиграем с этими ссылками. Создадим файл и жесткую ссылку на него

In [134]:
!rm -r /tmp/caos
!mkdir /tmp/caos
!echo "Hello, world!" > /tmp/caos/sample_file.txt
!cat /tmp/caos/sample_file.txt
!ln /tmp/caos/sample_file.txt /tmp/caos/sample_file_hard.txt
!ls -li /tmp/caos

Hello, world!
total 16
9294521 -rw-r--r--@ 2 k.afentev  wheel  14 Sep 27 11:31 sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  14 Sep 27 11:31 sample_file_hard.txt


inode соответствуют ожиданиям

In [135]:
!echo "Hello, world!" > /tmp/caos/other_sample_file.txt
!ls -li /tmp/caos

total 24
9294526 -rw-r--r--@ 1 k.afentev  wheel  14 Sep 27 11:31 other_sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  14 Sep 27 11:31 sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  14 Sep 27 11:31 sample_file_hard.txt


Изменения, внесенные в жесткую ссылку, отражены в оригинальном файле

In [136]:
!echo "\nHello from hardlink" >> /tmp/caos/sample_file_hard.txt
!cat /tmp/caos/sample_file.txt

Hello, world!

Hello from hardlink


Создадим символическую ссылку, у нее свой inode

In [137]:
!ln -s /tmp/caos/sample_file.txt /tmp/caos/sample_file_symlink.txt
!ls -li /tmp/caos

total 24
9294526 -rw-r--r--@ 1 k.afentev  wheel  14 Sep 27 11:31 other_sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  35 Sep 27 11:31 sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  35 Sep 27 11:31 sample_file_hard.txt
9294530 lrwxr-xr-x@ 1 k.afentev  wheel  25 Sep 27 11:31 sample_file_symlink.txt -> /tmp/caos/sample_file.txt


`cat` по символической ссылке вывел содержимое исходного файла, так как он "прошелся" по символической ссылке

In [138]:
!cat /tmp/caos/sample_file_symlink.txt

Hello, world!

Hello from hardlink


Изменения, внесенные по символической ссылке, отражены в оригинальном файле

In [139]:
!echo "\nAnd from symlink!" >> /tmp/caos/sample_file_symlink.txt
!cat /tmp/caos/sample_file.txt

Hello, world!

Hello from hardlink

And from symlink!


Переименуем оригинальный файл. Жесткая ссылка от этого не пострадает, а вот символическая сломается

In [140]:
!mv /tmp/caos/sample_file.txt /tmp/caos/moved_file.txt
!cat /tmp/caos/sample_file_hard.txt

Hello, world!

Hello from hardlink

And from symlink!


In [143]:
!cat /tmp/caos/sample_file_symlink.txt

cat: /tmp/caos/sample_file_symlink.txt: No such file or directory


In [142]:
!ls -li /tmp/caos

total 24
9294521 -rw-r--r--@ 2 k.afentev  wheel  54 Sep 27 11:31 moved_file.txt
9294526 -rw-r--r--@ 1 k.afentev  wheel  14 Sep 27 11:31 other_sample_file.txt
9294521 -rw-r--r--@ 2 k.afentev  wheel  54 Sep 27 11:31 sample_file_hard.txt
9294530 lrwxr-xr-x@ 1 k.afentev  wheel  25 Sep 27 11:31 sample_file_symlink.txt -> /tmp/caos/sample_file.txt


Попробуем поиграть с директориями:

Создать жесткую ссылку на директорию не получается:

In [152]:
!ln /tmp/caos /tmp/caos2.0

ln: /tmp/caos: Is a directory


Почему?

Символическую ссылку создать можно

In [153]:
!ln -s /tmp/caos /tmp/caos2.0symlink

In [161]:
!ls -H /tmp/caos2.0symlink
#   -H      Symbolic links on the command line are followed.

moved_file.txt          sample_file_hard.txt
other_sample_file.txt   sample_file_symlink.txt


На самом деле жесткие ссылки на директории бывают

<img src="academic_meme.webp" alt="This is all hypothetical isn't it, Tom?" width="400"/>

In [ ]:
!ls -lia /tmp/caos

total 24
9294519 drwxr-xr-x@   6 k.afentev  wheel    192 Sep 27 11:32 .
6277594 drwxrwxrwt  448 root       wheel  14336 Sep 27 11:38 ..
9294521 -rw-r--r--@   2 k.afentev  wheel     54 Sep 27 11:31 moved_file.txt
9294526 -rw-r--r--@   1 k.afentev  wheel     14 Sep 27 11:31 other_sample_file.txt
9294521 -rw-r--r--@   2 k.afentev  wheel     54 Sep 27 11:31 sample_file_hard.txt
9294530 lrwxr-xr-x@   1 k.afentev  wheel     25 Sep 27 11:31 sample_file_symlink.txt -> /tmp/caos/sample_file.txt


In [151]:
!ls -dlia /tmp/caos

9294519 drwxr-xr-x@ 6 k.afentev  wheel  192 Sep 27 11:32 /tmp/caos


`.` и `..` - это жесткие ссылки на директории.

Создадим пустую директорию:

In [ ]:
!rm tmp
!mkdir tmp
!stat -c %h tmp

rm: tmp: is a directory
mkdir: tmp: File exists
stat: illegal option -- c
usage: stat [-FLnq] [-f format | -l | -r | -s | -x] [-t timefmt] [file ...]


```c
2
```

In [165]:
!mkdir tmp/tmp2
!stat -c %h tmp

stat: illegal option -- c
usage: stat [-FLnq] [-f format | -l | -r | -s | -x] [-t timefmt] [file ...]


```c
3
```

### Вернемся к `stat`

In [171]:
!./basic_stat.out /tmp/caos/sample_file_symlink.txt

is regular: yes    is directory: no     is symbolic link: no 


Почему `stat` упал? Потому что симлинк провис. Исправим это и попробуем снова:

In [178]:
!mv /tmp/caos/moved_file.txt /tmp/caos/sample_file.txt
!./basic_stat.out /tmp/caos/sample_file_symlink.txt
!./basic_stat.out /tmp/caos2.0symlink

mv: /tmp/caos/moved_file.txt: No such file or directory
is regular: yes    is directory: no     is symbolic link: no 
is regular: no     is directory: yes    is symbolic link: no 


Что не так на этот раз? Дело в том, что `stat` проходит по симлинкам и возвращает информация сразу для целевого файла. Чтобы этого не происходило, есть `lstat`:

In [174]:
!cat basic_lstat.c
!gcc basic_lstat.c -o basic_lstat.out

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[]) {
    assert(argc > 1);

    struct stat s;
    int result = lstat(argv[1], &s);
    if (result != 0) {
        printf("lstat failed\n");
        return 1;
    }
    printf("is regular: %s    ", ((s.st_mode & S_IFMT) == S_IFREG) ? "yes" : "no "); // can use predefined mask
    printf("is directory: %s    ", S_ISDIR(s.st_mode) ? "yes" : "no "); // or predefined macro
    printf("is symbolic link: %s\n", S_ISLNK(s.st_mode) ? "yes" : "no "); 
    return 0;
}

In [177]:
!./basic_lstat.out /tmp/caos/sample_file_symlink.txt
!./basic_lstat.out /tmp/caos2.0symlink

is regular: no     is directory: no     is symbolic link: yes
is regular: no     is directory: no     is symbolic link: yes


Работает!

Попробуем через `stat` извлечь времена модификации и доступа к файлам. Для разнообразия будем использовать `fstat`, который принимает не путь к файлу, а файловый дескриптор

In [180]:
!cat access_time.c
!gcc access_time.c -o access_time.out

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>
#include <time.h>

int main(int argc, char *argv[]) {
    struct stat s;
    fstat(0, &s); 
    
    printf("update time: %s", asctime(gmtime(&s.st_mtimespec.tv_sec))); // '\n' есть в строке генерируемой asctime
    printf("access time: %s", asctime(gmtime(&s.st_atimespec.tv_sec)));

    return 0;
}

In [184]:
!./access_time.out < ../sem02-data-encoding/bool_is_1_bit.jpg # ассоциируем этот файл с 0-ым дескриптором (stdin)

update time: Sat Sep 20 06:01:48 2025
access time: Sat Sep 20 11:56:03 2025


`stat` может быть не подходить для работы с большими файлами из-за переполнения `int32`. На такие случаи есть `stat64`.

### Работа с директориями

Для работы с директориями есть функции `opendir`, `readir`, `mkdir` и другие

In [185]:
!cat my_ls.c
!gcc my_ls.c -o my_ls.out

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>
#include <dirent.h>

int main(int argc, char** argv) {
    assert(argc > 1);

    DIR *dir;
    struct dirent *ent;
    if ((dir = opendir(argv[1])) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            printf("%s\n", ent->d_name);
        }
        closedir(dir);
    } else {
        printf("Cannot open directory\n");
        return 1;
    }
}


In [188]:
!./my_ls.out /tmp/caos

.
..
sample_file.txt
sample_file_symlink.txt
other_sample_file.txt
sample_file_hard.txt


Также работать с директориями можно и с помощью `read` и флага `O_DIRECTORY`. Будет получен файловый дескриптор, с которым можно будет работать.

> Что будет, если вызвать `read` по этому файловому дескриптору?

## FUSE

[FUSE на вики](https://en.wikipedia.org/wiki/Filesystem_in_Userspace)

![FUSE](https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/FUSE_structure.svg/490px-FUSE_structure.svg.png)


`FUSE` - file system in user space. Возьмем `json` файл и примонтируем его в виде файловой системы.

In [193]:
!rm -r /tmp/json_mount
!mkdir /tmp/json_mount

In [194]:
!cat fuse/fuse_json.py

#!/usr/bin/env python3

import os
import sys
import json
import errno
from fuse import FUSE, FuseOSError, Operations

class JsonFs(Operations):
    def __init__(self, json_data):
        self.json_data = json_data
        self.files = {}
        self.dirs = {'/': []}
        self._build_tree('/', json_data)

    def _build_tree(self, path, data):
        if isinstance(data, dict):
            self.dirs[path] = list(data.keys())
            for key, value in data.items():
                new_path = os.path.join(path, key)
                if isinstance(value, (dict, list)):
                    self.dirs[new_path] = []
                    self._build_tree(new_path, value)
                else:
                    self.files[new_path] = str(value).encode('utf-8')
        elif isinstance(data, list):
            self.dirs[path] = [str(i) for i in range(len(data))]
            for i, item in enumerate(data):
                new_path = os.path.join(path, str(i))
                if isinstance(

In [196]:
!cat fuse/example.json

{
    "users": [
        {"name": "Alice", "age": 30},
        {"name": "Bob", "age": 25}
    ],
    "settings": {
        "debug": true,
        "level": "info"
    }
}

In [199]:
!python3 fuse/fuse_json.py fuse/example.json /tmp/json_mount

Traceback (most recent call last):
  File "/Users/k.afentev/CAOS/caos-2025/sem03-lowlevel-io-fs/fuse/fuse_json.py", line 7, in <module>
    from fuse import FUSE, FuseOSError, Operations
ModuleNotFoundError: No module named 'fuse'


In [200]:
!tree /tmp/json_mount/ 

zsh:1: command not found: tree


```bash
.
├── settings
│   ├── debug
│   └── level
└── users
    ├── 0
    │   ├── age
    │   └── name
    └── 1
        ├── age
        └── name

5 directories, 6 files
```

*Замечание*: на самом деле `open`, `write`, `stat` и т. д. в том виде, в котором мы их рассматривали сегодня, не являются сисколами сами по себе. Это обертки, написанные на С, над настоящими сисколами. Мы еще вернемся к этой теме на последующих семинарах